# Create and Run chat flows using Promptflow Python SDK

### Overview

Prompt flow is a suite of development tools designed to streamline the end-to-end development cycle of LLM-based AI applications, from ideation, prototyping, testing, evaluation to production deployment and monitoring. It makes prompt engineering much easier and enables you to build LLM apps with production quality.

In this hands-on, you will be able to:
Create flows that link fine-tuned phi3.5 endpoint(Python code) and gpt model in a executable workflow.
Debug and iterate your flows, especially tracing interaction with LLMs with ease.

#### 1. Set up Promptflow client with Credential and configuration

#### 2. Create a new chat flow by providing the flow name and description.

#### 3. Run Basic Promptflow with questions to compare models

#### 4. Run Context Added Promptflow with the outdoor questions

#### 5. Use serverless endpoint to run the Promptflow with context

[Note] Please use `Python 3.10 - SDK v2 (azureml_py310_sdkv2)` conda environment.


In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
lab_prep_dir = os.getcwd().split("SLMWorkshopCN")[0] + "SLMWorkshopCN/0_lab_preparation"
sys.path.append(os.path.abspath(lab_prep_dir))

from common import check_kernel
check_kernel()

Kernel: python31014jvsc74a57bd01f90a0206bde5cf3732dab79adbbcc7570d5fab64b89fc69d46a8fe33664a709


In [2]:
import json
import os
import time

# Import required libraries
from promptflow.azure import PFClient
from promptflow.entities import Run
# Import required libraries
from azure.identity import DefaultAzureCredential, EnvironmentCredential, InteractiveBrowserCredential
from dotenv import load_dotenv
from azure.core.exceptions import HttpResponseError
load_dotenv()
with open('./config_prd.json', 'r') as f:
    config = json.load(f)
    
print(config["subscription_id"])
print(config["resource_group"])
print(config["workspace_name"]) # Azure AI Foundry project name which is not the same as the Azure ML workspace name

49aee8bf-3f02-464f-a0ba-e3467e7d85e2
rg-slmwrkshp_9
slmwrkshp9


In [4]:
from tqdm import tqdm

# Monitor the status of the run_result
def monitor_status(pf_azure_client:PFClient, run_result:Run):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = pf_azure_client.runs.get(run_result).status
        if status == "Preparing":
            pbar.update(1)
        while status != "Completed" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = pf_azure_client.runs.get(run_result).status
        pbar.update(1)
        print("Promptflow Running Completed")

## 1. Set up Promptflow client with Credential and configuration

-   Create a promptflow client with the credential and configuration. You need to set the `config_prd.json` file with subscription_id, resource_group and workspace_name


In [5]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()
# if you cannot use DefaultAzureCredential and InteractiveBrowserCredential you need to set up the Managed identity in your .env file

pf_azure_client = PFClient.from_config(credential=credential, path="./config_prd.json")

# pf_azure_client = PFClient(credential=credential, 
#                            subscription_id="your subscription id", 
#                            resource_group_name="your resource group name", 
#                            workspace_name="your workspace name")            

try:
    workspace = pf_azure_client.ml_client.workspaces.get(name=config["workspace_name"])
    print(f"Connected to Azure AI Foundry Workspace: {workspace.name}")
    print(f"Workspace Location: {workspace.location}")
    print(f"Workspace ID: {workspace.id}")
except HttpResponseError as e:
    print(f"Failed to connect to Azure ML Workspace: {e}")


Found the config file in: config_prd.json


Connected to Azure AI Foundry Workspace: slmwrkshp9
Workspace Location: eastus
Workspace ID: /subscriptions/49aee8bf-3f02-464f-a0ba-e3467e7d85e2/resourceGroups/rg-slmwrkshp_9/providers/Microsoft.MachineLearningServices/workspaces/slmwrkshp9


## 2. Create a new chat flow by providing the flow name and description.

-   Create a new chat flow by providing the flow name and description. You can view and clone the flow on Azure AI Foundry UI.

> ✨ **_important_** <br>
> Grant the Storage File Data Privileged Contributor, Storage Blob Data Contributor at the storage of AI Foundryrole to user, group, service principle and managed Identity which you are trying to access, you can execute the code below.


### Check the exist connections

-   currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.
    > ✨ **_important_** <br>
    > Check your connection information in Azure AI Foundry Management Center


In [6]:
from jinja2 import Environment, FileSystemLoader
from pathlib import Path

env = Environment(loader=FileSystemLoader('.'))

# Read the template file
template = env.get_template('./flow-template/chat.flow.dag.yaml')

# Define the variables for the template
your_phi35_connection_name = "slmwrkshp_phi35"
your_gpt4o_connection_name = "cog-pgwgybluulpec"
variables = {
	"your_phi35_connection_name": your_phi35_connection_name,
	"your_gpt4o_connection_name": your_gpt4o_connection_name,
}

rendered_content = template.render(variables)
Path('./chat/flow.dag.yaml').write_text(rendered_content)

print(Path('./chat/flow.dag.yaml').read_text()) 

$schema: https://azuremlschemas.azureedge.net/promptflow/latest/Flow.schema.json
environment:
  python_requirements_txt: requirements.txt
inputs:
  question:
    type: string
    is_chat_input: true
    default: What is the capital of France?
outputs:
  phi35_answer:
    type: string
    reference: ${phi35.output}
    is_chat_output: false
  gpt4o_answer:
    type: string
    reference: ${gpt4o.output}
    is_chat_output: true
nodes:
- name: phi35
  type: python
  source:
    type: code
    path: phi35_finetuned.py
  inputs:
    connection: slmwrkshp_phi35
    input_data: ${inputs.question}
- name: gpt4o
  type: llm
  source:
    type: code
    path: chat.jinja2
  inputs:
    deployment_name: gpt-4o
    temperature: 0.7
    top_p: 1
    max_tokens: 256
    question: ${inputs.question}
  connection: cog-pgwgybluulpec
  api: chat
  module: promptflow.tools.aoai
  use_variants: false


In [7]:
#Grant the right accessibility to create the flow 
pf_azure_client.flows.create_or_update(flow="chat/", type="chat", display_name="comparison flow created from python sdk", description="fine-tuned model comparison flow")#AI Foundry Prj/Prompt Flow/Flows标签页中可看到

Flow created successfully:
{
    "name": "c2f5542c-62b3-4a24-8242-63e7f9e72264",
    "type": "chat",
    "description": "fine-tuned model comparison flow",
    "path": "Users/luogang/promptflow/chat-02-19-2025-22-18-05/flow.dag.yaml",
    "code": "azureml://locations/eastus/workspaces/7d935d47-d75a-46d0-967a-9034bdea1de5/flows/c2f5542c-62b3-4a24-8242-63e7f9e72264",
    "display_name": "comparison flow created from python sdk",
    "owner": {
        "user_object_id": "a0490c2e-4f6d-490a-9f03-0b49b8a3c480",
        "user_tenant_id": "16b3c013-d300-468d-ac64-7eda0820b6d3",
        "user_name": "Gang Luo"
    },
    "is_archived": false,
    "created_date": "2025-02-19 14:18:20.812505+00:00",
    "flow_portal_url": "https://ai.azure.com/projectflows/c2f5542c-62b3-4a24-8242-63e7f9e72264/7d935d47-d75a-46d0-967a-9034bdea1de5/details/Flow?wsid=/subscriptions/49aee8bf-3f02-464f-a0ba-e3467e7d85e2/resourcegroups/rg-slmwrkshp_9/providers/Microsoft.MachineLearningServices/workspaces/slmwrkshp9"
}


## 3. Run Basic Promptflow with questions to compare models

-   Run the Promptflow with the simple questions such as "What is the capital of France?" and compare the results of the models


In [8]:
flow_path = "./chat"
data_path = "./data/questions_basic.jsonl"

column_mapping = {
    "question": "${data.question}"
}

run_result = pf_azure_client.run(#AI Foundry Prj/Prompt Flow/Runs标签页中可看到，这里代码提交的run与上面pf_azure_client.flows.create_or_update创建的flow并无关联，只是丢个xml上去跑而已
    flow=flow_path,
    type="chat",
    data=data_path,
    column_mapping=column_mapping,
    display_name="chat_with_data",
    tags={"chat_with_jsonl": "", "1st_round": ""},
)


Uploading chat (0.01 MBs): 100%|██████████| 5677/5677 [00:01<00:00, 4763.31it/s]


Uploading questions_basic.jsonl (< 1 MB): 100%|██████████| 724/724 [00:00<00:00, 2.54kB/s]


[2025-02-19 22:18:37 +0800][promptflow][WARNING] - You're using compute session, if it's first time you're using it, it may take a while to build session and you may see 'NotStarted' status for a while. 
[2025-02-19 22:18:37 +0800][promptflow][WARNING] - The trace Cosmos DB for current workspace/project is not ready yet, your traces might not be logged and stored properly.
To enable it, please run `pf config set trace.destination=azureml://subscriptions/<subscription-id>/resourceGroups/<resource-group-name>/providers/Microsoft.MachineLearningServices/workspaces/<workspace-or-project-name>`, prompt flow will help to get everything ready.



Portal url: https://ai.azure.com/projectflows/trace/run/chat_variant_0_20250219_221831_260866/details?wsid=/subscriptions/49aee8bf-3f02-464f-a0ba-e3467e7d85e2/resourcegroups/rg-slmwrkshp_9/providers/Microsoft.MachineLearningServices/workspaces/slmwrkshp9


In [9]:
monitor_status(pf_azure_client, run_result)

Running Status:   0%|          | 0/3 [00:00<?, ?step/s]

Current Status: NotStarted


Running Status:   0%|          | 0/3 [00:06<?, ?step/s]


KeyboardInterrupt: 

Retrying due to transient client side error HTTPSConnectionPool(host='westus2-0.in.applicationinsights.azure.com', port=443): Max retries exceeded with url: /v2.1/track (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fc0b26556c0>: Failed to resolve 'westus2-0.in.applicationinsights.azure.com' ([Errno -3] Temporary failure in name resolution)")).
Retrying due to transient client side error HTTPSConnectionPool(host='westus2-0.in.applicationinsights.azure.com', port=443): Max retries exceeded with url: /v2.1/track (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fc0b27d9ff0>: Failed to resolve 'westus2-0.in.applicationinsights.azure.com' ([Errno -3] Temporary failure in name resolution)")).
Retrying due to transient client side error HTTPSConnectionPool(host='westus2-0.in.applicationinsights.azure.com', port=443): Max retries exceeded with url: /v2.1/track (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection o

In [ ]:
detail = pf_azure_client.get_details(run_result)

detail

---

下面4节其实就是把数据换了一下，5节其实就是把flow.dag.yaml模板换了下，做法基本和前面相似，可选择略过

---

## 4. Run Context Added Promptflow with the outdoor questions

-   Run the Promptflow using the context data and ask the outdoor product related questions to compare the results of the models


### Check the exist connections

-   currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.
    > ✨ **_important_** <br>
    > Check your connection information in Azure AI Foundry Management Center


In [ ]:
from jinja2 import Environment, FileSystemLoader
from pathlib import Path

env = Environment(loader=FileSystemLoader('.'))

# Read the template file
template = env.get_template('./flow-template/chat-context.flow.dag.yaml')

# Define the variables for the template
variables = {
	"your_phi35_connection_name": your_phi35_connection_name,
	"your_gpt4o_connection_name": your_gpt4o_connection_name
}

rendered_content = template.render(variables)
Path('./chat-context/flow.dag.yaml').write_text(rendered_content)

print(Path('./chat-context/flow.dag.yaml').read_text()) 

In [ ]:
flow_path = "./chat-context"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    #context_simple.json中只有固定的一条ctx所以可以这样写
}

run_result_with_context = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_context_data",
    tags={"chat_with_context_jsonl": "", "1st_round": ""},
)


In [ ]:
monitor_status(pf_azure_client, run_result_with_context)

In [ ]:
detail = pf_azure_client.get_details(run_result_with_context)

detail

## 5. Use serverless endpoint to run the Promptflow with context

-   Create a serverless endpoint to run the Promptflow with the context. You can use the endpoint to run the flow with the context.


### deploy your serverless endpoint

-   go to the Azure AI Foundry > Model catalog > search phi-3.5 > deply Phi-3.5-mini-instruct as your serverless endpint
    <br>
    ![serverless endpoint](./images/deploy_serverless_endpoint.jpg)
    <br>
    <br>
-   once the deployment is done, go to Deployments and you can see the endpoint deployed in the endpoint section. Click to check the details and copy key and phi35-mini-instruct: Chat Completion endpoint url
    ![copy connection](./images/copy_connection.jpg)
    <br>
    <br>
-   go to Settings in Azure AI Foundry > Connections > create a new connection naming phi35-serverless with the copied key and endpoint url
    ![create new serverless connection](./images/create_new_serverless_connection.jpg)


### Check the exist connections

-   currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.
    > ✨ **_important_** <br>
    > Check your connection information in Azure AI Foundry Management Center


In [ ]:
from jinja2 import Environment, FileSystemLoader
from pathlib import Path

env = Environment(loader=FileSystemLoader('.'))

# Read the template file
template = env.get_template('./flow-template/chat-serverless.flow.dag.yaml')

# Define the variables for the template with your connection names for chat serverless 
variables = {
	"your_phi35_serverless_connection_name": "replace with your connection name",
	"your_gpt4o_connection_name": "replace with your connection name"
}

rendered_content = template.render(variables)
Path('./chat-serverless/flow.dag.yaml').write_text(rendered_content)

print(Path('./chat-serverless/flow.dag.yaml').read_text()) 

In [ ]:
flow_path = "./chat-serverless"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

run_serverless_result = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_serverless_context_data",
    tags={"chat_serverless_context_jsonl": "", "1st_round": ""},
)


In [ ]:
monitor_status(pf_azure_client, run_serverless_result)

In [ ]:
detail = pf_azure_client.get_details(run_serverless_result)

detail

In [ ]:
context.get("context")   